# Model Examples

In [14]:
import numpy as np
import torch
import torch.nn as nn

### Manual Regression

In [9]:
X = np.array([1,2,3,4], dtype=np.float32) #input
Y = np.array([2,4,6,8], dtype=np.float32) #output

w = 0.0 #weights

def forward (x):
    return w * x

def loss(y, y_pred):
    return ((y_pred-y)**2).mean()

#gradient of loss
# MSE = 1/N * (w*x -y)**2
# dJ/dw = 1/N * 2x * (w*x - y)

def gradient(x,y,y_pred):
    return np.dot(2*x, y_pred-y).mean()

print(f'Prediction before training: f(5) = {forward(5):.3f}')

#training

learning_rate = 0.01
n_iters = 20

for epoch in range(n_iters):
    # prediction = forward pass
    y_pred = forward(X)

    # loss
    l = loss(Y, y_pred)

    # gradients
    dw = gradient(X,Y,y_pred)

    # update weights
    w -= learning_rate * dw

    if epoch % 5 == 0:
        print(f'epoch {epoch +1}: w = {w:.3f}, loss = {l:.8f}')

print(f'Prediction after training: f(5) = {forward(5):.3f}')


Prediction before training: f(5) = 0.000
epoch 1: w = 1.200, loss = 30.00000000
epoch 6: w = 1.992, loss = 0.00314570
epoch 11: w = 2.000, loss = 0.00000033
epoch 16: w = 2.000, loss = 0.00000000
Prediction after training: f(5) = 10.000


### Automatic Gradient Calculations

In [12]:
X = torch.tensor([1,2,3,4], dtype=torch.float32) #input
Y = torch.tensor([2,4,6,8], dtype=torch.float32) #output

w = torch.tensor(0.0, dtype = torch.float32, requires_grad=True) #weights

def forward (x):
    return w * x

def loss(y, y_pred):
    return ((y_pred-y)**2).mean()

print(f'Prediction before training: f(5) = {forward(5):.3f}')

#training

learning_rate = 0.01
n_iters = 100

for epoch in range(n_iters):
    # prediction = forward pass
    y_pred = forward(X)

    # loss
    l = loss(Y, y_pred)

    # gradients = backward pass
    l.backward()

    # update weights
    with torch.no_grad():
        w -= learning_rate * w.grad
    
    # zero gradients
    w.grad.zero_()

    if epoch % 10 == 0:
        print(f'epoch {epoch +1}: w = {w:.3f}, loss = {l:.8f}')

print(f'Prediction after training: f(5) = {forward(5):.3f}')


Prediction before training: f(5) = 0.000
epoch 1: w = 0.300, loss = 30.00000000
epoch 11: w = 1.665, loss = 1.16278565
epoch 21: w = 1.934, loss = 0.04506890
epoch 31: w = 1.987, loss = 0.00174685
epoch 41: w = 1.997, loss = 0.00006770
epoch 51: w = 1.999, loss = 0.00000262
epoch 61: w = 2.000, loss = 0.00000010
epoch 71: w = 2.000, loss = 0.00000000
epoch 81: w = 2.000, loss = 0.00000000
epoch 91: w = 2.000, loss = 0.00000000
Prediction after training: f(5) = 10.000


### Automatic Loss and parameter update with torch classes

In [17]:
X = torch.tensor([[1],[2],[3],[4]], dtype=torch.float32) #input
Y = torch.tensor([[2],[4],[6],[8]], dtype=torch.float32) #output

n_samples, n_features = X.shape
print(n_samples, n_features)

X_test = torch.tensor([5], dtype=torch.float32)
input_size = n_features
output_size = n_features

# Pre-implemented model
# model = nn.Linear(input_size, output_size)

# Custom model

class LinearRegression(nn.Module):
    
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        # define layers
        self.lin = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.lin(x)

model = LinearRegression(input_size, output_size)

print(f'Prediction before training: f(5) = {model(X_test).item():.3f}')

#training

learning_rate = 0.01
n_iters = 100

loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

for epoch in range(n_iters):
    # prediction = forward pass
    y_pred = model(X)

    # loss
    l = loss(Y, y_pred)

    # gradients = backward pass
    l.backward()

    # update weights
    optimizer.step()
    
    # zero gradients
    optimizer.zero_grad()

    if epoch % 10 == 0:
        [w, b] = model.parameters()
        print(f'epoch {epoch +1}: w = {w[0][0].item():.3f}, loss = {l:.8f}')

print(f'Prediction after training: f(5) = {model(X_test).item():.3f}')

4 1
Prediction before training: f(5) = 0.665
epoch 1: w = 0.346, loss = 25.03364944
epoch 11: w = 1.498, loss = 0.76098931
epoch 21: w = 1.690, loss = 0.12640263
epoch 31: w = 1.728, loss = 0.10377308
epoch 41: w = 1.740, loss = 0.09733783
epoch 51: w = 1.749, loss = 0.09166202
epoch 61: w = 1.756, loss = 0.08632654
epoch 71: w = 1.763, loss = 0.08130188
epoch 81: w = 1.770, loss = 0.07656978
epoch 91: w = 1.777, loss = 0.07211296
Prediction after training: f(5) = 9.553


## Feedforward Example

In [18]:
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#hyper parameters
input_size = 784 # 28x28
hidden_size = 100 # semi-arbitrary
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

# MNIST
train_dataset = torchvision.datasets.MNIST(root='./data')